In [ ]:
"""
from https://github.com/aiforsec/InsiderThreat
"""

In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import numpy as np
import dask.dataframe as dd

TOPIC_NUM = 100

tmp_dir = Path('./')
answers_dir = Path(r"C:\ITD_Project\src\DANTE\answers")
answers_file = answers_dir/"insiders.csv"
dataset_version = '5.2'
assert(answers_file.is_file())

In [ ]:
# https://stackoverflow.com/questions/57531388/how-can-i-reduce-the-memory-of-a-pandas-dataframe
def reduce_mem_usage(df, ignore_cols = None ):
    """
    iterate through all the columns of a dataframe and
    modify the data type to reduce memory usage.
    """
    if ignore_cols is None:
	    ignore_cols = []

    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype
        print(col, col_type)

        if col in ignore_cols:
            continue

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max <\
                  np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <\
                   np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <\
                   np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <\
                   np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max <\
                   np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max <\
                   np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
        print("\tNew dtype:  ", df[col].dtype)
    end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}'
                              'MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem)
                                             / start_mem))

    return df

In [ ]:
preprocessed_dir = Path(r"C:\ITD_Project\_output")
assert(preprocessed_dir.is_dir())

In [ ]:
processed_files = ['logon_preprocessed.csv', 'http_preprocessed.csv','device_preprocessed.csv',
                   'email_preprocessed.csv','file_preprocessed.csv']
columns = ['id', 'date', 'user', 'is_usual_pc', 'is_work_time', 'subtype']
temp_df = []

for file in processed_files:
    temp_df.append(pd.read_csv(preprocessed_dir/file,
                   usecols = columns))

In [ ]:
df = pd.concat(temp_df, axis=0)
df.isna().sum()

id              0
date            0
user            0
is_usual_pc     0
is_work_time    0
subtype         0
dtype: int64

In [ ]:
for temp in temp_df:
    del temp

In [ ]:
df = reduce_mem_usage(df, ignore_cols=['id', 'date'])

id object
date object
user object
	New dtype:   category
is_usual_pc bool
	New dtype:   float16
is_work_time bool
	New dtype:   float16
subtype object
	New dtype:   category
Memory usage after optimization is: 2360.95MB
Decreased by 26.2%


In [ ]:
df.head()

c:\ITD_Project\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,id,date,user,is_usual_pc,is_work_time,subtype
0,{Q4D5-W4HH44UC-5188LWZK},2010-01-02 02:24:51,JBI1134,0.0,0.0,Logon
1,{G7V0-S4TP95SA-9203AOGR},2010-01-02 02:38:28,JBI1134,0.0,0.0,Logoff
2,{B4U7-K4DB84LM-2657VBFY},2010-01-02 04:55:52,JBI1134,0.0,0.0,Logon
3,{K2E1-W7VG04OA-3686THSV},2010-01-02 05:02:28,JBI1134,0.0,0.0,Logoff
4,{Z7B2-Z8AS75YE-4485NAFE},2010-01-02 06:35:00,HMI1448,1.0,0.0,Logon


In [ ]:
subtype_encoder = LabelEncoder()
type_encoder = LabelEncoder()
df.subtype = df.subtype.map(str)
df['subtype'] = subtype_encoder.fit_transform(df['subtype'])
#df['type'] = type_encoder.fit_transform(df['type'])

df['action_id'] = df.is_usual_pc.astype(np.int8) * 100 + df.is_work_time.astype(np.int8) * 10 + df.subtype
df['date'] = pd.to_datetime(df['date'], format='ISO8601')

df = df[['id', 'date', 'user', 'action_id']]

df.to_pickle(str(tmp_dir / "df.pkl"))
del df

In [ ]:
content_dir = Path(r'C:\ITD_Project\_output')
content_file = ['email_lda.csv', 'file_lda.csv', 'http_lda.csv']
content_cols = ['id', 'content']
temp_df = []

for file in content_file:
    df = pd.read_csv(content_dir/ file, usecols = content_cols)
    df = reduce_mem_usage(df, ignore_cols=content_cols)
    temp_df.append(df)

id object
content object
Memory usage after optimization is: 264.94MB
Decreased by 0.0%
id object
content object
Memory usage after optimization is: 13.55MB
Decreased by 0.0%
id object
content object
Memory usage after optimization is: 274.53MB
Decreased by 0.0%


In [ ]:
content_df = pd.concat(temp_df, axis=0)

In [ ]:
for df in temp_df:
    del df

In [ ]:
content_df = reduce_mem_usage(content_df, ignore_cols=['id', 'content'])
content_df.to_csv(str(tmp_dir / 'content_df.csv'))

id object
content object
Memory usage after optimization is: 829.53MB
Decreased by 0.0%


In [ ]:
content_df = pd.read_csv(r"C:\ITD_Project\src\DANTE\content_df.csv").set_index('id').drop('Unnamed: 0', axis=1)
df = pd.read_pickle(str(tmp_dir / "df.pkl"))

# Merge the csv files.
df = pd.merge(content_df, df, how='inner', on=['id'])

In [ ]:
df = df.reset_index().drop(['index'], axis=1)
df['day'] = df.date.dt.floor('D')
df.set_index('date')
df.head()

,id,content,date,user,action_id,day
0,{N9X0-P3SX99UT-3623QQUN},"[(15, 0.96746325)]",2010-01-02 06:49:35,KMC1934,106,2010-01-02
1,{F5G8-U4KL36AB-1579CZGX},"[(70, 0.9266499), (89, 0.04112698)]",2010-01-02 06:50:35,KMC1934,106,2010-01-02
2,{U8D2-M5FH65BW-3939CQIC},"[(60, 0.5426919), (65, 0.440649)]",2010-01-02 06:53:37,KMC1934,106,2010-01-02
3,{A5X6-X3KN62LU-5816QTBZ},"[(25, 0.31295696), (29, 0.665325)]",2010-01-02 06:53:46,KMC1934,106,2010-01-02
4,{Q8R5-A2WJ20NG-9108MVZX},"[(5, 0.27211964), (74, 0.6663706), (87, 0.0373...",2010-01-02 06:54:33,KMC1934,106,2010-01-02


In [ ]:
action_id_lists = df.groupby(['user', 'day'], sort=True)\
    ['action_id'].apply(list)

content_lists = df.groupby(['user', 'day'], sort=True)\
    ['content'].apply(list)

action_id_lists = action_id_lists.reset_index()
content_lists = content_lists.reset_index()

df_merged = pd.merge(action_id_lists, content_lists, how='inner', on=['user', 'day'])
df_merged.to_csv(str(tmp_dir / "merged_df.csv"), index=False)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_24892\122020583.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  action_id_lists = df.groupby(['user', 'day'], sort=True)\
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_24892\122020583.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  content_lists = df.groupby(['user', 'day'], sort=True)\


In [ ]:
main_df = pd.read_csv(answers_file)
main_df = main_df[main_df['dataset'].astype(str) ==
                  str(dataset_version)].drop(['dataset', 'details'], axis=1)

In [ ]:
df = pd.read_csv(str(tmp_dir / "merged_df.csv"), usecols=['action_id', 'user', 'day']) # changed action to action_id, and added user and day
df_m = df.merge(main_df, left_on='user', right_on='user', how='left')
print(df_m.head())
df = df_m.drop(['start', 'end', 'day', 'user'], axis=1)
df.to_csv(str(tmp_dir / 'merged_answers_df.csv'), index=False)

      user         day                                          action_id  \
0  AAB1302  2010-01-02  [116, 116, 117, 116, 116, 116, 116, 116, 116, ...   
1  AAB1302  2010-01-03                                                NaN   
2  AAB1302  2010-01-04  [116, 117, 116, 116, 116, 116, 116, 116, 117, ...   
3  AAB1302  2010-01-05  [117, 116, 117, 116, 106, 106, 106, 106, 114, ...   
4  AAB1302  2010-01-06  [116, 116, 117, 116, 117, 116, 116, 117, 117, ...   

   scenario start  end  
0       NaN   NaN  NaN  
1       NaN   NaN  NaN  
2       NaN   NaN  NaN  
3       NaN   NaN  NaN  
4       NaN   NaN  NaN  


In [ ]:
df.head()

,action_id,scenario
0,"[116, 116, 117, 116, 116, 116, 116, 116, 116, ...",NaN
1,NaN,NaN
2,"[116, 117, 116, 116, 116, 116, 116, 116, 117, ...",NaN
3,"[117, 116, 117, 116, 106, 106, 106, 106, 114, ...",NaN
4,"[116, 116, 117, 116, 117, 116, 116, 117, 117, ...",NaN


In [ ]:
df = pd.read_csv(str(tmp_dir / 'merged_df.csv'),) # changed merged_answers_df to merged_df
df = reduce_mem_usage(df, ignore_cols=['action_id', 'content'])

user object
	New dtype:   category
day object
	New dtype:   category
action_id object
content object
Memory usage after optimization is: 19.82MB
Decreased by 37.2%


In [ ]:
df.head()

,user,day,action_id,content
0,AAB1302,2010-01-02,"[116, 116, 117, 116, 116, 116, 116, 116, 116, ...","['[(21, 0.015315825), (44, 0.819448), (70, 0.1..."
1,AAB1302,2010-01-03,NaN,NaN
2,AAB1302,2010-01-04,"[116, 117, 116, 116, 116, 116, 116, 116, 117, ...","['[(64, 0.9652026)]', '[(23, 0.4497046), (25, ..."
3,AAB1302,2010-01-05,"[117, 116, 117, 116, 106, 106, 106, 106, 114, ...","['[(53, 0.9711812)]', '[(1, 0.18867843), (59, ..."
4,AAB1302,2010-01-06,"[116, 116, 117, 116, 117, 116, 116, 117, 117, ...","['[(9, 0.65201116), (39, 0.3287789)]', '[(33, ..."


In [ ]:
df_ = df.sort_values(['user', 'day']) # updated df to df_ here (for merged.pkl file)
#df_ = df_m.reset_index()#.drop(['date'], axis=1)
df_ = df_.groupby(['user', 'day'], sort=False, as_index=True).agg(list)
df_.head()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_24892\823337152.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_ = df_.groupby(['user', 'day'], sort=False, as_index=True).agg(list)


action_id  \
user    day                                                             
AAB1302 2010-01-02  [[116, 116, 117, 116, 116, 116, 116, 116, 116,...   
        2010-01-03                                              [nan]   
        2010-01-04  [[116, 117, 116, 116, 116, 116, 116, 116, 117,...   
        2010-01-05  [[117, 116, 117, 116, 106, 106, 106, 106, 114,...   
        2010-01-06  [[116, 116, 117, 116, 117, 116, 116, 117, 117,...   

                                                              content  
user    day                                                            
AAB1302 2010-01-02  [['[(21, 0.015315825), (44, 0.819448), (70, 0....  
        2010-01-03                                              [nan]  
        2010-01-04  [['[(64, 0.9652026)]', '[(23, 0.4497046), (25,...  
        2010-01-05  [['[(53, 0.9711812)]', '[(1, 0.18867843), (59,...  
        2010-01-06  [['[(9, 0.65201116), (39, 0.3287789)]', '[(33,...

In [ ]:
df_.to_pickle("merged.pkl")

In [ ]:
df_.head()

,action_id,scenario
0,"[116, 116, 117, 116, 116, 116, 116, 116, 116, ...",NaN
1,NaN,NaN
2,"[116, 117, 116, 116, 116, 116, 116, 116, 117, ...",NaN
3,"[117, 116, 117, 116, 106, 106, 106, 106, 114, ...",NaN
4,"[116, 116, 117, 116, 117, 116, 116, 117, 117, ...",NaN


In [ ]:
import ast
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix

for idx, row in df.iterrows():
    content_raw = row.content
    action_id_raw = row.action_id

    # --- Skip missing or already-parsed rows safely ---
    if isinstance(content_raw, (float, np.floating)) and np.isnan(content_raw):
        continue
    if isinstance(action_id_raw, (float, np.floating)) and np.isnan(action_id_raw):
        continue

    # --- Parse strings safely ---
    try:
        parsed_content = (
            ast.literal_eval(content_raw)
            if isinstance(content_raw, str)
            else content_raw
        )
        parsed_action_id = (
            ast.literal_eval(action_id_raw)
            if isinstance(action_id_raw, str)
            else action_id_raw
        )
    except (ValueError, SyntaxError):
        continue

    # --- Skip invalid parsed results ---
    if not isinstance(parsed_content, list):
        continue

    # --- Build sparse matrix ---
    mat = csc_matrix((len(parsed_content), TOPIC_NUM), dtype=float)

    for action_idx, action in enumerate(parsed_content):
        if isinstance(action, str):
            try:
                action = ast.literal_eval(action)
            except (ValueError, SyntaxError):
                continue

        for topic, probability in action:
            mat[action_idx, topic] = probability

    # --- Store back ---
    df.at[idx, "content"] = mat
    df.at[idx, "action_id"] = parsed_action_id

# --- Save ---
'''df.to_pickle("final.pkl)'''


c:\ITD_Project\Lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


'df.to_pickle("final.pkl)'

In [ ]:
df.head()

,action_id,scenario
0,"[116, 116, 117, 116, 116, 116, 116, 116, 116, ...",NaN
1,NaN,NaN
2,"[116, 117, 116, 116, 116, 116, 116, 116, 117, ...",NaN
3,"[117, 116, 117, 116, 106, 106, 106, 106, 114, ...",NaN
4,"[116, 116, 117, 116, 117, 116, 116, 117, 117, ...",NaN


In [ ]:
df['scenario'].unique()

array([nan,  3.,  4.,  1.,  2.])